##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 이미지 분류를 위한 적대적 정규화

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">}TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/neural-structured-learning/g3doc/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

## 개요

이 튜토리얼에서는 Neural Structured Learning(NSL) 프레임워크를 사용하여 이미지 분류에 적대적 학습([Goodfellow et al., 2014](https://arxiv.org/abs/1412.6572))을 사용하는 방법을 살펴봅니다.

적대적 학습의 핵심 아이디어는 유기적 훈련 데이터와 함께 적대적으로 교란된 데이터(적대적 예라고 함)로 모델을 훈련하는 것입니다. 적대적인 예는 의도적으로 모델을 잘못된 예측 또는 분류로 오도하기 위해 구성됩니다. 이러한 예를 사용한 훈련을 통해 모델은 예측할 때 적대적 교란에 대해 견고함을 유지하도록 학습합니다.

이 튜토리얼에서는 Neural Structured Learning 프레임워크를 사용하여 견고한 모델을 얻기 위해 적대적 학습을 적용하는 다음 절차를 설명합니다.

1. 기본 모델로 신경망을 만듭니다. 이 튜토리얼에서는 `tf.keras` 함수형 API를 사용하여 기본 모델을 생성합니다. 이 절차는 `tf.keras` 순차 및 하위 클래스화 API에 의해서 생성된 모델과 호환됩니다. TensorFlow의 Keras 모델에 대한 자세한 내용은 이 [ 설명서](https://www.tensorflow.org/api_docs/python/tf/keras/Model)를 참조하세요.
2. NSL 프레임워크에서 제공하는 **`AdversarialRegularization`** 래퍼 클래스로 기본 모델을 래핑하여 새 `tf.keras.Model` 인스턴스를 만듭니다. 이 새로운 모델은 훈련 목표에서 정규화 항으로 적대적 손실을 포함합니다.
3. 훈련 데이터의 예를 특성 사전으로 변환합니다.
4. 새 모델을 훈련하고 평가합니다.

## Recap for Beginners

There is a corresponding [video explanation](https://youtu.be/Js2WJkhdU7k) on adversarial learning for image classification part of the TensorFlow Neural Structured Learning Youtube series. Below, we have summarized the key concepts explained in this video, expanding on the explanation provided in the Overview section above.

The NSL framework jointly optimizes both image features and structured signals to help neural networks better learn. However, what if there is no explicit structure available to train the neural network? This tutorial explains one approach involving the creation of adversarial neighbors (modified from the original sample) to dynamically construct a structure.

Firstly, adversarial neighbors are defined as modified versions of the sample image applied with small perturbations that mislead a neural net into outputting inaccurate classifications. These carefully designed perturbations are typically based on the reverse gradient direction and are meant to confuse the neural net during training. Humans may not be able to tell the difference between a sample image and it's generated adversarial neighbor. However, to the neural net, the applied perturbations are effective at leading to an inaccurate conclusion.

그런 다음 생성된 적대적 이웃은 샘플에 연결되어 끝을 이어가며 동적으로 구조를 구성합니다. 이러한 연결을 통해 신경망이 오분류로 인한 혼란을 피하면서 샘플과 적대적 이웃 간의 유사성을 유지하는 방법을 학습함에 따라 전체 신경망의 품질과 정확성이 개선됩니다.

The code segment below is a high-level explanation of the steps involved while the rest of this tutorial goes into further depth and technicality.

1. 데이터를 읽고 준비합니다. MNIST 데이터세트를 로드하고 기능 값을 정규화하여 [0,1] 범위를 유지합니다.

```
import neural_structured_learning as nsl

(x_train, y_train), (x_train, y_train) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
```

1. 신경망을 구축합니다. 이 예에서는 Sequential Keras 기본 모델을 사용합니다.

```
model = tf.keras.Sequential(...)
```


1. Configure the adversarial model. Including the hyperparameters: multiplier applied on the adversarial regularization, empirically chosen differ values for step size/learning rate. Invoke adversarial regularization with a wrapper class around the constructed neural network.

```
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config)
```

1. 표준 Keras 워크플로(컴파일, 맞춤, 평가)로 마무리합니다.

```
adv_model.compile(optimizer='adam', loss='sparse_categorizal_crossentropy', metrics=['accuracy'])
adv_model.fit({'feature': x_train, 'label': y_train}, epochs=5)
adv_model.evaluate({'feature': x_test, 'label': y_test})
```

What you see here is adversarial learning enabled in 2 steps and 3 simple lines of code. This is the simplicity of the neural structured learning framework. In the following sections, we expand upon this procedure.

## 설정

Neural Structured Learning 패키지를 설치합니다.

In [ ]:
!pip install --quiet neural-structured-learning

라이브러리를 가져옵니다. `neural_structured_learning`을 `nsl`로 축약합니다.

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## 하이퍼 매개변수

모델 훈련 및 평가를 위해 하이퍼 매개변수(`HParams` 객체에서)를 수집하고 설명합니다.

입출력:

- **`input_shape`**: 입력 텐서의 형상, 각 이미지는 1채널의 28x28 픽셀입니다.
- **`num_classes`**: 총 10개의 클래스가 있으며 10자리 [0-9]에 해당합니다.

모델 아키텍처:

- **`conv_filters`**: 각각 컨볼루셔널 레이어에서 필터의 수를 지정하는 숫자의 목록입니다.
- **`kernel_size`**: 모든 컨볼루셔널 레이어가 공유하는 2D 컨볼루션 창의 크기입니다.
- **`pool_size`**: 각 최대 풀링 레이어에서 이미지를 축소하는 요소입니다.
- **`num_fc_units`**: 완전히 연결된 각 레이어의 단위(예: 너비) 수입니다.

훈련 및 평가:

- **`batch_size`**: 훈련 및 평가에 사용되는 배치 크기입니다.
- **`epochs`**: 훈련 epoch의 수입니다.

적대적 학습:

- **`adv_multiplier`**: 훈련 목표에서 레이블이 지정된 손실에 대한 적대적 손실의 가중치입니다.
- **`adv_step_size`** : 적대적 교란의 크기입니다.
- **`adv_grad_norm`**: 적대적 교란의 크기를 측정하기 위한 표준입니다.


In [ ]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## MNIST 데이터세트

[MNIST 데이터세트](http://yann.lecun.com/exdb/mnist/)에는 손으로 쓴 숫자('0'에서 '9'까지)의 회색조 이미지가 포함됩니다. 각 이미지는 저해상도(28x28 픽셀)에서 한 자리를 표시합니다. 관련된 작업은 이미지를 숫자당 하나씩 10개의 범주로 분류하는 것입니다.

여기에 [TensorFlow 데이터세트](https://www.tensorflow.org/datasets)에서 MNIST 데이터세트를 로드합니다. 데이터 다운로드 및 `tf.data.Dataset` 구성을 처리합니다. 로드된 데이터세트에는 두 개의 하위 집합이 있습니다.

- 60,000개의 예제로 `train`
- 10,000개의 예제로 `test`.

두 하위 집합의 예는 다음 두 키를 사용하여 특성 사전에 저장됩니다.

- `image`: 픽셀 값 배열, 0~255 범위.
- `label`: Groundtruth 레이블, 범위는 0~9입니다.

In [ ]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

모델을 수치적으로 안정적으로 만들기 위해 `normalize` 함수에 데이터세트를 매핑하여 픽셀 값을 [0, 1]로 정규화합니다. 훈련 세트를 셔플하고 일괄 처리한 후 기본 모델의 훈련을 위해 예제를 특성 튜플 `(image, label)`로 변환합니다. 또한, 나중에 사용할 수 있도록 튜플에서 사전으로 변환하는 함수를 제공합니다.

In [ ]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

## 기본 모델

사용할 기본 모델은 세개의 컨볼루셔널 레이어와 두 개의 완전 연결 레이어(`HPARAMS` 정의됨)로 구성된 신경망입니다. 여기에서는 Keras 함수형 API를 사용하여 이를 정의합니다. 다른 API 또는 모델 아키텍처(예: 하위 클래스화)도 자유롭게 사용하세요. NSL 프레임워크는 세 가지 유형의 Keras API를 모두 지원합니다.

In [ ]:
def build_base_model(hparams):
  """Builds a model according to the architecture defined in `hparams`."""
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)

  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(
            x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling between convolutional layers
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes, activation='softmax')(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [ ]:
base_model = build_base_model(HPARAMS)
base_model.summary()

다음으로 기본 모델을 훈련하고 평가합니다.

In [ ]:
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                   metrics=['acc'])
base_model.fit(train_dataset, epochs=HPARAMS.epochs)

In [ ]:
results = base_model.evaluate(test_dataset)
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

기본 모델이 테스트세트에서 99% 정확성을 달성하는 것을 볼 수 있습니다. 아래의 [적대적 교란에서의 견고성](#scrollTo=HXK9MGG8lBX3)에서 얼마나 견고한지 확인할 수 있습니다.

## 적대적 정규화 모델

여기에서는 NSL 프레임워크를 사용하여 몇 줄의 코드로 적대적 훈련을 Keras 모델에 통합하는 방법을 보여줍니다. 기본 모델은 훈련 목표에 적대적 정규화를 포함하는 새로운 `tf.Keras.Model`을 만들기 위해 래핑됩니다.

먼저, 도우미 함수 `nsl.configs.make_adv_reg_config`를 사용하여 모든 관련 하이퍼 매개변수가 포함된 config 객체를 만듭니다.

In [ ]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

이제 `AdversarialRegularization`으로 기본 모델을 래핑할 수 있습니다. 여기에서 새로운 기본 모델 (`base_adv_model`)을 생성하여 기존 모델(`base_model`)을 나중에 비교하는 데 사용할 수 있습니다.

반환된 `adv_model`은 `tf.keras.Model` 객체이며, 훈련 목표에는 적대적 손실에 대한 정규화 항이 포함됩니다. 손실을 계산하려면 모델이 일반 입력(특성 `image`) 외에도 레이블 정보(특성 `label`)에 액세스할 수 있어야 합니다. 이러한 이유로 데이터세트의 예를 튜플에서 다시 사전으로 변환합니다. 그리고 `label_keys` 매개변수를 통해 레이블 정보가 포함된 특성을 모델에 알립니다.

In [ ]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

다음으로 적대적 정규화 모델을 컴파일, 훈련 및 평가합니다. "손실 사전에서 누락된 출력"과 같은 경고가 있을 수 있으며, `adv_model`이 총 손실을 계산하기 위해 기본 구현에 의존하지 않기 때문에 괜찮습니다.

In [ ]:
adv_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                   metrics=['acc'])
adv_model.fit(train_set_for_adv_model, epochs=HPARAMS.epochs)

In [ ]:
results = adv_model.evaluate(test_set_for_adv_model)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

적대적 정규화 모델도 테스트세트에서 매우 우수한 것을 볼 수 있습니다(99% 정확성).

## 적대적 교란에서의 견고성

이제 적대적 교란에서의 견고성을 위해 기본 모델과 적대적 정규화 모델을 비교합니다.

`AdversarialRegularization.perturb_on_batch` 함수를 사용하여 적대적으로 교란된 예를 생성합니다. 그리고 기본 모델을 기반으로 한 세대를 사용합니다. 이를 위해 기본 모델을 `AdversarialRegularization`으로 래핑합니다. 훈련(`Model.fit`)을 호출하지 않는 한, 모델에서 학습된 변수는 변경되지 않으며, 모델은 여전히 [기본 모델](#scrollTo=JrrMpPNmpCKK) 섹션에서와 같습니다.

In [ ]:
reference_model = nsl.keras.AdversarialRegularization(
    base_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config)
reference_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['acc'])

평가할 모델을 사전에 수집하고 각 모델에 대한 메트릭 객체를 만듭니다.

기본 모델과 같은 입력 형식(레이블 정보 필요 없음)을 갖기 위해 `adv_model.base_model`을 사용합니다. `adv_model.base_model`에서 학습된 변수는 `adv_model`에서의 변수와 같습니다.

In [ ]:
models_to_eval = {
    'base': base_model,
    'adv-regularized': adv_model.base_model
}
metrics = {
    name: tf.keras.metrics.SparseCategoricalAccuracy()
    for name in models_to_eval.keys()
}

다음은 교란된 예를 생성하고 이를 사용하여 모델을 평가하는 루프입니다. 다음 섹션에서 시각화를 위해 교란된 이미지, 레이블 및 예측값을 저장합니다.

In [ ]:
perturbed_images, labels, predictions = [], [], []

for batch in test_set_for_adv_model:
  perturbed_batch = reference_model.perturb_on_batch(batch)
  # Clipping makes perturbed examples have the same range as regular ones.
  perturbed_batch[IMAGE_INPUT_NAME] = tf.clip_by_value(                          
      perturbed_batch[IMAGE_INPUT_NAME], 0.0, 1.0)
  y_true = perturbed_batch.pop(LABEL_INPUT_NAME)
  perturbed_images.append(perturbed_batch[IMAGE_INPUT_NAME].numpy())
  labels.append(y_true.numpy())
  predictions.append({})
  for name, model in models_to_eval.items():
    y_pred = model(perturbed_batch)
    metrics[name](y_true, y_pred)
    predictions[-1][name] = tf.argmax(y_pred, axis=-1).numpy()

for name, metric in metrics.items():
  print('%s model accuracy: %f' % (name, metric.result().numpy()))

입력이 적대적으로 교란될 때 기본 모델의 정확성이 급격히 떨어지는 것을 볼 수 있습니다(99%에서 약 50%로). 반면에 적대적 정규화 모델의 정확성은 약간만 저하됩니다(99%에서 95%로). 이것은 모델의 견고성 향상에 대한 적대적 학습의 효과를 보여줍니다.

## 적대적으로 교란된 이미지의 예

다음에서 적대적으로 교란된 이미지를 살펴봅니다. 교란된 이미지는 여전히 사람이 인식할 수 있는 숫자를 보여주지만, 기본 모델을 속일 수 있음을 알 수 있습니다.

In [ ]:
batch_index = 0

batch_image = perturbed_images[batch_index]
batch_label = labels[batch_index]
batch_pred = predictions[batch_index]

batch_size = HPARAMS.batch_size
n_col = 4
n_row = (batch_size + n_col - 1) / n_col

print('accuracy in batch %d:' % batch_index)
for name, pred in batch_pred.items():
  print('%s model: %d / %d' % (name, np.sum(batch_label == pred), batch_size))

plt.figure(figsize=(15, 15))
for i, (image, y) in enumerate(zip(batch_image, batch_label)):
  y_base = batch_pred['base'][i]
  y_adv = batch_pred['adv-regularized'][i]
  plt.subplot(n_row, n_col, i+1)
  plt.title('true: %d, base: %d, adv: %d' % (y, y_base, y_adv))
  plt.imshow(tf.keras.preprocessing.image.array_to_img(image), cmap='gray')
  plt.axis('off')

plt.show()

## 결론

Neural Structured Learning(NSL) 프레임워크를 사용하여 이미지 분류를 위한 적대적 학습을 사용하는 방법을 시연했습니다. 사용자가 다양한 적대적 설정(하이퍼 매개변수에서)을 실험하고 모델 견고성에 어떻게 영향을 미치는지 확인할 것을 권장합니다.